In [1]:
import cobra
import pandas as pd
import re
import numpy as np
import scipy.stats as st
from matplotlib import pyplot as plt
from pathlib import Path
import sys
sys.path.append('../../code/')
import leakage, utils
import pubchempy as pcp
import seaborn as sns

In [2]:
model = cobra.io.read_sbml_model('../../models/e_coli/momentiJO1366.xml')
model.reactions.DHAPT.knock_out()
model.reactions.THRA.knock_out()
model.reactions.THRA2.knock_out()
model.reactions.F6PA_TG_forward.knock_out()
# Speculations
# model.reactions.PFL.knock_out()
#I'm more and more conviced that PFL is not active in aerobic conditions
# https://doi.org/10.1016/j.jbc.2021.101423

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-26


In [3]:
solution = model.optimize()
print(model.summary())

Objective
1.0 BIOMASS_Ec_iJO1366_core_53p95M = 0.6619602089406109

Uptake
------
Metabolite     Reaction      Flux  C-Number  C-Flux
 prot_pool  ER_pool_TG_     0.095         0   0.00%
     ca2_e     EX_ca2_e  0.003446         0   0.00%
      cl_e      EX_cl_e  0.003446         0   0.00%
 cobalt2_e EX_cobalt2_e 1.655E-05         0   0.00%
     cu2_e     EX_cu2_e 0.0004693         0   0.00%
     fe2_e     EX_fe2_e   0.01063         0   0.00%
  glc__D_e  EX_glc__D_e     12.45         6 100.00%
       k_e       EX_k_e    0.1292         0   0.00%
     mg2_e     EX_mg2_e  0.005743         0   0.00%
     mn2_e     EX_mn2_e 0.0004574         0   0.00%
    mobd_e    EX_mobd_e 8.539E-05         0   0.00%
     nh4_e     EX_nh4_e      7.15         0   0.00%
     ni2_e     EX_ni2_e 0.0002138         0   0.00%
      o2_e      EX_o2_e     16.23         0   0.00%
      pi_e      EX_pi_e    0.6385         0   0.00%
     so4_e     EX_so4_e     0.167         0   0.00%
     zn2_e     EX_zn2_e 0.0002257  

In [8]:
sp_dict = {}
for m in model.metabolites:
    if m.compartment =='c':
        if m.formula_weight and m.formula_weight < 350:
            raw_sp = solution.shadow_prices[m.id]
            calculated_sp = leakage.estimate_shadow_price_for_met(model, m, solution, delta = 0.1)
            sp_dict[m.id] = [raw_sp, calculated_sp]

/Users/ssulheim/opt/anaconda3/lib/python3.9/site-packages/cobra/core/metabolite.py:191: UserWarning: The element 'R' does not appear in the periodic table
  warn(f"The element {e} does not appear in the periodic table")
/Users/ssulheim/opt/anaconda3/lib/python3.9/site-packages/cobra/core/metabolite.py:191: UserWarning: The element 'X' does not appear in the periodic table
  warn(f"The element {e} does not appear in the periodic table")


In [9]:
sp_df = pd.DataFrame(sp_dict).T
sp_df.columns = ['From solver', 'Brute force']

In [10]:
sp_df['Diff'] = (sp_df['From solver']-sp_df['Brute force']).abs()
sel = sp_df.loc[sp_df['Diff'] > 1e-2, :]

In [11]:
sel.sort_values(by='Diff')

,From solver,Brute force,Diff
adn_c,-0.252721,-0.263289,0.010568
ade_c,-0.243588,-0.256033,0.012445
2h3oppan_c,-0.012052,-0.024575,0.012523
hpyr_c,-0.012568,-0.025091,0.012523
glyc__R_c,-0.012691,-0.025214,0.012523
rml1p_c,-0.007216,-0.019775,0.012559
dha_c,0.000000,-0.014167,0.014167
2mcit_c,-0.018296,-0.032647,0.014351
fc1p_c,-0.006197,-0.020795,0.014598
ppa_c,-0.000240,-0.015276,0.015036
